
## Notebook 07: Regularización L1, L2 y ElasticNet (Clasificación)
### **Objetivo**: Comparar Ridge (L2), Lasso (L1) y ElasticNet usando Regresión Logística para predecir contratos de "Alto Impacto" (Top 25% del mercado).


In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.sql.functions import col, when
import pandas as pd
import numpy as np

# %%
spark = SparkSession.builder \
    .appName("SECOP_Regularizacion_Logistica") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()


 ###  Carga y Binarización (Percentil 75)
 Transformamos el problema de regresión a clasificación binaria.


In [4]:

df_raw = spark.read.parquet("/opt/spark-data/processed/secop_ml_ready.parquet")

# Usamos 'label' (que tiene el valor numérico) para crear los cuartiles
discretizer = QuantileDiscretizer(
    numBuckets=4, 
    inputCol="label", 
    outputCol="cuartil"
)

df_cuartiles = discretizer.fit(df_raw).transform(df_raw)

# Clase 1: Top 25% (cuartil 3.0). Clase 0: Resto.
df_final = df_cuartiles.withColumnRenamed("label", "valor_original") \
                       .withColumn("label", when(col("cuartil") == 3.0, 1.0).otherwise(0.0)) \
                       .select("features", "label")

train, test = df_final.randomSplit([0.7, 0.3], seed=42)

print(f"Registros entrenamiento: {train.count():,}")
print(f"Registros prueba: {test.count():,}")

Registros entrenamiento: 309,240
Registros prueba: 132,708



## RETO 1: Entender la Regularización

**Respuesta**: El escenario de AUC train alto y AUC test bajo indica **Overfitting**. 
La regularización ayuda penalizando la complejidad del modelo, evitando que se ajuste a ruidos específicos de los datos de entrenamiento.



## RETO 2: Configurar el Evaluador

**Objetivo**: Crear un evaluador para clasificación.


In [5]:
# Usamos areaUnderROC (AUC) como métrica principal para clasificación binaria
evaluator = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)


## RETO 3: Experimento de Regularización
Probaremos combinaciones de λ (regParam) y α (elasticNetParam).



In [6]:
reg_params = [0.0, 0.01, 0.1, 1.0]
elastic_params = [0.0, 0.5, 1.0] # 0.0=Ridge, 1.0=Lasso

print(f"Combinaciones totales: {len(reg_params) * len(elastic_params)}")

# %%
resultados = []

for reg in reg_params:
    for elastic in elastic_params:
        # 1. Configurar Regresión Logística
        lr = LogisticRegression(
            featuresCol="features",
            labelCol="label",
            maxIter=50,
            regParam=reg,
            elasticNetParam=elastic
        )

        # 2. Entrenar
        model = lr.fit(train)
        
        # 3. Evaluar
        predictions = model.transform(test)
        auc_test = evaluator.evaluate(predictions)
        auc_train = model.summary.areaUnderROC

        # Determinar tipo de regularización
        if reg == 0.0: reg_type = "Base (Sin Reg)"
        elif elastic == 0.0: reg_type = "Ridge (L2)"
        elif elastic == 1.0: reg_type = "Lasso (L1)"
        else: reg_type = "ElasticNet"

        resultados.append({
            "regParam": reg,
            "elasticNetParam": elastic,
            "tipo": reg_type,
            "auc_test": auc_test,
            "auc_train": auc_train
        })

        print(f"{reg_type:20s} | λ={reg:5.2f} | α={elastic:.1f} | AUC Test: {auc_test:.4f}")

Combinaciones totales: 12


Base (Sin Reg)       | λ= 0.00 | α=0.0 | AUC Test: 0.8267


Base (Sin Reg)       | λ= 0.00 | α=0.5 | AUC Test: 0.8267


Base (Sin Reg)       | λ= 0.00 | α=1.0 | AUC Test: 0.8266
Ridge (L2)           | λ= 0.01 | α=0.0 | AUC Test: 0.8267


ElasticNet           | λ= 0.01 | α=0.5 | AUC Test: 0.8239


Lasso (L1)           | λ= 0.01 | α=1.0 | AUC Test: 0.8221
Ridge (L2)           | λ= 0.10 | α=0.0 | AUC Test: 0.8262
ElasticNet           | λ= 0.10 | α=0.5 | AUC Test: 0.7976
Lasso (L1)           | λ= 0.10 | α=1.0 | AUC Test: 0.7913


Ridge (L2)           | λ= 1.00 | α=0.0 | AUC Test: 0.8219


ElasticNet           | λ= 1.00 | α=0.5 | AUC Test: 0.5000


Lasso (L1)           | λ= 1.00 | α=1.0 | AUC Test: 0.5000



## RETO 4: Analizar Resultados

**Objetivo**: Identificar el modelo con mayor AUC en el set de prueba.



In [7]:

df_resultados = pd.DataFrame(resultados)
mejor_modelo_data = df_resultados.loc[df_resultados['auc_test'].idxmax()]

print("\n--- RESULTADOS ORDENADOS ---")
print(df_resultados.sort_values(by="auc_test", ascending=False).to_string(index=False))

print(f"\nGANADOR: {mejor_modelo_data['tipo']} con AUC={mejor_modelo_data['auc_test']:.4f}")



--- RESULTADOS ORDENADOS ---
 regParam  elasticNetParam           tipo  auc_test  auc_train
     0.00              0.5 Base (Sin Reg)  0.826729   0.826985
     0.00              0.0 Base (Sin Reg)  0.826670   0.826985
     0.01              0.0     Ridge (L2)  0.826658   0.827009
     0.00              1.0 Base (Sin Reg)  0.826645   0.827015
     0.10              0.0     Ridge (L2)  0.826217   0.826276
     0.01              0.5     ElasticNet  0.823898   0.823577
     0.01              1.0     Lasso (L1)  0.822112   0.821761
     1.00              0.0     Ridge (L2)  0.821869   0.822013
     0.10              0.5     ElasticNet  0.797608   0.795743
     0.10              1.0     Lasso (L1)  0.791319   0.789208
     1.00              0.5     ElasticNet  0.500000   0.500000
     1.00              1.0     Lasso (L1)  0.500000   0.500000

GANADOR: Base (Sin Reg) con AUC=0.8267



 ## RETO 5: Comparar Overfitting

**Objetivo**: Analizar la brecha (Gap) entre Train y Test.


In [9]:
df_resultados['gap'] = df_resultados['auc_train'] - df_resultados['auc_test']
print("\n--- ANÁLISIS DE BRECHA (OVERFITTING) ---")
print(df_resultados[['tipo', 'regParam', 'gap']].sort_values(by='gap'))



--- ANÁLISIS DE BRECHA (OVERFITTING) ---
              tipo  regParam       gap
8       Lasso (L1)      0.10 -0.002111
7       ElasticNet      0.10 -0.001865
5       Lasso (L1)      0.01 -0.000351
4       ElasticNet      0.01 -0.000321
10      ElasticNet      1.00  0.000000
11      Lasso (L1)      1.00  0.000000
6       Ridge (L2)      0.10  0.000059
9       Ridge (L2)      1.00  0.000144
1   Base (Sin Reg)      0.00  0.000256
0   Base (Sin Reg)      0.00  0.000315
3       Ridge (L2)      0.01  0.000351
2   Base (Sin Reg)      0.00  0.000370


## RETO 6: Entrenar y Guardar Modelo Final



In [11]:
lr_final = LogisticRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=100,
    regParam=mejor_modelo_data['regParam'],
    elasticNetParam=mejor_modelo_data['elasticNetParam']
)

modelo_final = lr_final.fit(train)
modelo_final.write().overwrite().save("/opt/spark-data/processed/modelo_logistico_final")
print(f"\n✓ Modelo guardado exitosamente.")

## RETO BONUS: Efecto Lasso (Sparsity)

for reg in [0.01, 0.1, 1.0]:
    m = LogisticRegression(regParam=reg, elasticNetParam=1.0).fit(train)
    coefs = m.coefficients.toArray()
    zeros = np.sum(np.abs(coefs) < 1e-6)
    print(f"Lasso λ={reg:5.2f} | Variables eliminadas: {zeros} de {len(coefs)}")

# %% [markdown]
# ## Preguntas de Reflexión
# 1. **¿Ridge vs Lasso?**: Ridge si todas las variables son importantes; Lasso para simplificar el modelo.
# 2. **¿regParam gigante?**: Genera Underfitting (el modelo es demasiado simple).
# 3. **¿Modelo base mejor?**: Solo si el gap con train es casi cero y el rendimiento es alto.

# %%
spark.stop()


✓ Modelo guardado exitosamente.
Lasso λ= 0.01 | Variables eliminadas: 49 de 64
Lasso λ= 0.10 | Variables eliminadas: 62 de 64


Lasso λ= 1.00 | Variables eliminadas: 64 de 64


la información importante para predecir contratos de alto valor está concentrada en unas pocas variables clave

El ganador fue el modelo sin regularización (regParam=0.0)